<a href="https://colab.research.google.com/github/anthonyibr24/Feature-selection-code/blob/breast_cancer/Breast_Cancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.datasets import load_breast_cancer
import pandas as pd
import numpy as np

data_set=load_breast_cancer()

#load dataset
data=load_breast_cancer()
df=pd.DataFrame(data.data,columns=data.feature_names)
df['target']=data.target
df.head()

#scale mean=0 std=1
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
df=df.drop('target',axis=1)
df[data.feature_names]=scaler.fit_transform(df[data.feature_names])

df['target']=data.target
df.describe().round(3)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
count,569.000,569.000,569.000,569.000,569.000,569.000,569.000,569.000,569.000,569.000,...,569.000,569.000,569.000,569.000,569.000,569.000,569.000,569.000,569.000,569.000
mean,-0.000,-0.000,-0.000,-0.000,0.000,-0.000,-0.000,0.000,-0.000,-0.000,...,0.000,-0.000,0.000,-0.000,-0.000,0.000,-0.000,-0.000,0.000,0.627
std,1.001,1.001,1.001,1.001,1.001,1.001,1.001,1.001,1.001,1.001,...,1.001,1.001,1.001,1.001,1.001,1.001,1.001,1.001,1.001,0.484
min,-2.030,-2.229,-1.985,-1.454,-3.112,-1.610,-1.115,-1.262,-2.744,-1.820,...,-2.224,-1.693,-1.222,-2.683,-1.444,-1.306,-1.745,-2.161,-1.602,0.000
25%,-0.689,-0.726,-0.692,-0.667,-0.711,-0.747,-0.744,-0.738,-0.703,-0.723,...,-0.749,-0.690,-0.642,-0.691,-0.681,-0.757,-0.756,-0.642,-0.692,0.000
50%,-0.215,-0.105,-0.236,-0.295,-0.035,-0.222,-0.342,-0.398,-0.072,-0.178,...,-0.044,-0.286,-0.341,-0.047,-0.270,-0.218,-0.223,-0.127,-0.216,1.000
75%,0.469,0.584,0.500,0.364,0.636,0.494,0.526,0.647,0.531,0.471,...,0.658,0.540,0.358,0.598,0.540,0.531,0.713,0.450,0.451,1.000
max,3.971,4.652,3.976,5.251,4.771,4.568,4.244,3.928,4.485,4.911,...,3.886,4.287,5.930,3.955,5.113,4.701,2.686,6.046,6.847,1.000


In [ ]:
df.shape

(569, 31)

In [ ]:
print(df['target'].value_counts())

target
1    357
0    212
Name: count, dtype: int64


# Full features

In [ ]:
#Cross validation
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression

lr=LogisticRegression()

cv_results = cross_validate(
    lr,
    df.drop('target', axis=1),
    df['target'],
    cv=4,
    scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'],
    return_train_score=False
)

print("\nAverage Metrics Across 4 Folds:")
print(f"Accuracy:  {np.mean(cv_results['test_accuracy']):.4f} ± {np.std(cv_results['test_accuracy']):.4f}")
print(f"Precision: {np.mean(cv_results['test_precision_macro']):.4f} ± {np.std(cv_results['test_precision_macro']):.4f}")
print(f"Recall:    {np.mean(cv_results['test_recall_macro']):.4f} ± {np.std(cv_results['test_recall_macro']):.4f}")
print(f"F1 Score:  {np.mean(cv_results['test_f1_macro']):.4f} ± {np.std(cv_results['test_f1_macro']):.4f}")


Average Metrics Across 4 Folds:
Accuracy:  0.9772 ± 0.0058
Precision: 0.9773 ± 0.0056
Recall:    0.9741 ± 0.0087
F1 Score:  0.9755 ± 0.0063


# Mutual Gain

In [ ]:
#Mutual gain
from sklearn.datasets import load_wine
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

#load dataset
data=data_set
df=pd.DataFrame(data.data,columns=data.feature_names)
df['target']=data.target
df.head()

X=df.drop('target',axis=1)
y=df['target']



In [ ]:
#Pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', SelectKBest(mutual_info_classif, k=15)),
    ('classifier', LogisticRegression())
])
pipeline.verbose=True

#Cross-validate and calculate all metrics
scores = cross_validate(
    pipeline,
    X, y,
    cv=4,
    scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
)

print("\nAverage Scores Across 4 Folds:")
for metric in ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']:
    mean_score = scores[f'test_{metric}'].mean()
    std_score = scores[f'test_{metric}'].std()
    print(f"{metric.capitalize()}: {mean_score:.4f} ± {std_score:.4f}")

[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.1s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.1s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.1s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.1s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s

Average Scores Across 4 Folds:
Accuracy: 0.9509 ± 0.0237
Precision_macro: 0.9491 ± 0.0260
Recall_macro: 0.9455 ± 0.0249
F1_macro: 0.9472 ± 0.0254


In [ ]:
#ANOVA
from sklearn.datasets import load_wine
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline


#load dataset
data=data_set
df=pd.DataFrame(data.data,columns=data.feature_names)
df['target']=data.target
df.head()

X=df.drop('target',axis=1)
y=df['target']

#Pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', SelectKBest(score_func=f_classif, k=15)),
    ('classifier', LogisticRegression())
])
pipeline.verbose=True

#Cross-validate and calculate all metrics
scores = cross_validate(
    pipeline,
    X, y,
    cv=4,
    scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
)

print("\nAverage Scores Across 4 Folds:")
for metric in ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']:
    mean_score = scores[f'test_{metric}'].mean()
    std_score = scores[f'test_{metric}'].std()
    print(f"{metric.capitalize()}: {mean_score:.4f} ± {std_score:.4f}")

[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.0s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.1s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.0s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.0s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.0s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s

Average Scores Across 4 Folds:
Accuracy: 0.9509 ± 0.0237
Precision_macro: 0.9491 ± 0.0260
Recall_macro: 0.9455 ± 0.0249
F1_macro: 0.9472 ± 0.0254


# Wrapper Methods

# Forward

In [ ]:
#FORWARD
from sklearn.datasets import load_wine
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier

#load dataset
data=data_set
df=pd.DataFrame(data.data,columns=data.feature_names)
df['target']=data.target
df.head()

X=df.drop('target',axis=1)
y=df['target']

knn=KNeighborsClassifier(5)



In [ ]:
#Pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', SequentialFeatureSelector(knn, n_features_to_select=15, direction='forward', scoring='accuracy')),
    ('classifier', LogisticRegression())
])
pipeline.verbose=True

#Cross-validate and calculate all metrics
scores = cross_validate(
    pipeline,
    X, y,
    cv=4,
    scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
)

print("\nAverage Scores Across 4 Folds:")
for metric in ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']:
    mean_score = scores[f'test_{metric}'].mean()
    std_score = scores[f'test_{metric}'].std()
    print(f"{metric.capitalize()}: {mean_score:.4f} ± {std_score:.4f}")


[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=  11.9s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=  11.5s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=  11.7s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=  13.6s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.2s

Average Scores Across 4 Folds:
Accuracy: 0.9684 ± 0.0036
Precision_macro: 0.9684 ± 0.0084
Recall_macro: 0.9652 ± 0.0044
F1_macro: 0.9661 ± 0.0035


# BACKWARD

In [ ]:
#BACKWARD
from sklearn.datasets import load_wine
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier

#load dataset
data=data_set
df=pd.DataFrame(data.data,columns=data.feature_names)
df['target']=data.target
df.head()

X=df.drop('target',axis=1)
y=df['target']

knn=KNeighborsClassifier(5)

In [ ]:
#Pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', SequentialFeatureSelector(knn, n_features_to_select=15, direction='backward', scoring='accuracy')),
    ('classifier', LogisticRegression())
])
pipeline.verbose=True

#Cross-validate and calculate all metrics
scores = cross_validate(
    pipeline,
    X, y,
    cv=4,
    scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
)

print("\nAverage Scores Across 4 Folds:")
for metric in ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']:
    mean_score = scores[f'test_{metric}'].mean()
    std_score = scores[f'test_{metric}'].std()
    print(f"{metric.capitalize()}: {mean_score:.4f} ± {std_score:.4f}")

[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   4.6s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   4.5s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   4.9s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   4.5s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s

Average Scores Across 4 Folds:
Accuracy: 0.9772 ± 0.0058
Precision_macro: 0.9772 ± 0.0035
Recall_macro: 0.9741 ± 0.0093
F1_macro: 0.9755 ± 0.0064


# RFE

In [ ]:
#RFE
from sklearn.datasets import load_wine
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE
from sklearn.svm import SVC

#load dataset
data=data_set
df=pd.DataFrame(data.data,columns=data.feature_names)
df['target']=data.target
df.head()

X=df.drop('target',axis=1)
y=df['target']



In [ ]:
#Pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', RFE(estimator=SVC(kernel='linear'), n_features_to_select=15)),
    ('classifier', LogisticRegression())
])
pipeline.verbose=True

#Cross-validate and calculate all metrics
scores = cross_validate(
    pipeline,
    X, y,
    cv=4,
    scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
)

print("\nAverage Scores Across 4 Folds:")
for metric in ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']:
    mean_score = scores[f'test_{metric}'].mean()
    std_score = scores[f'test_{metric}'].std()
    print(f"{metric.capitalize()}: {mean_score:.4f} ± {std_score:.4f}")

[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.1s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.1s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.1s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.1s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.1s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s

Average Scores Across 4 Folds:
Accuracy: 0.9719 ± 0.0070
Precision_macro: 0.9719 ± 0.0059
Recall_macro: 0.9680 ± 0.0096
F1_macro: 0.9698 ± 0.0076


# Embeded Methods

# Random Forest

In [ ]:
#RF
from sklearn.datasets import load_wine
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel


#load dataset
data=data_set
df=pd.DataFrame(data.data,columns=data.feature_names)
df['target']=data.target
df.head()

X=df.drop('target',axis=1)
y=df['target']


In [ ]:
#Pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', SelectFromModel(RandomForestClassifier(random_state=42), max_features=15)),
    ('classifier', LogisticRegression())
])
pipeline.verbose=True

#Cross-validate and calculate all metrics
scores = cross_validate(
    pipeline,
    X, y,
    cv=4,
    scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
)

print("\nAverage Scores Across 4 Folds:")
for metric in ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']:
    mean_score = scores[f'test_{metric}'].mean()
    std_score = scores[f'test_{metric}'].std()
    print(f"{metric.capitalize()}: {mean_score:.4f} ± {std_score:.4f}")

[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.3s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.3s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.3s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.1s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.4s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s

Average Scores Across 4 Folds:
Accuracy: 0.9544 ± 0.0234
Precision_macro: 0.9535 ± 0.0273
Recall_macro: 0.9493 ± 0.0223
F1_macro: 0.9511 ± 0.0247


# Logistic using L1 penalty

In [ ]:
#LR L1 penalty
from sklearn.datasets import load_wine
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest


#load dataset
data=data_set
df=pd.DataFrame(data.data,columns=data.feature_names)
df['target']=data.target
df.head()

X=df.drop('target',axis=1)
y=df['target']


In [ ]:
#Pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', SelectFromModel(LogisticRegression(penalty='l1',solver='saga',C=0.1,max_iter=3000,random_state=42),max_features=15)),
    ('classifier', LogisticRegression())
])
pipeline.verbose=True

#Cross-validate and calculate all metrics
scores = cross_validate(
    pipeline,
    X, y,
    cv=4,
    scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
)

print("\nAverage Scores Across 4 Folds:")
for metric in ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']:
    mean_score = scores[f'test_{metric}'].mean()
    std_score = scores[f'test_{metric}'].std()
    print(f"{metric.capitalize()}: {mean_score:.4f} ± {std_score:.4f}")

[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.3s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.1s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.7s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.1s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.3s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.4s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s

Average Scores Across 4 Folds:
Accuracy: 0.9631 ± 0.0058
Precision_macro: 0.9627 ± 0.0090
Recall_macro: 0.9591 ± 0.0057
F1_macro: 0.9604 ± 0.0060
